In [1]:
import gym
import mobile_env

from ray import tune
from ray.tune.registry import register_env

pygame 2.0.1 (SDL 2.0.14, Python 3.8.11)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# TODO: install stable-baselines; ray rllib; ... AND also mcom

### Multi-Agent Control Setting with RLlib:

First, register the environment: ....

In [3]:
# env = gym.make("mobile-small-v0")
# obs = env.reset()

# def register(name):
#     import mobile_env
#     if name == 'mobile-small-v0':
#         return gym.make("mobile-small-v0")

#     print(name)
#     raise NotImplementedError
    
# register_env("mobile-small-v0", register)

Run the MA agent:

In [4]:
# tune.run(
#     "APEX_DDPG",
#     stop={"episodes_total": 60000},
#     checkpoint_freq=10,
#     config={
#         # Enviroment specific
#         "env": "mobile-small-v0",
#         # General
#         "num_gpus": 0,
#         "num_workers": 2,
            
#         # Method specific.
#         "multiagent": {
#             # We only have one policy (calling it "shared").
#             # Class, obs/act-spaces, and config will be derived
#             # automatically.
#             "policies": {"shared_policy"},
#             # Always use "shared" policy.
#             "policy_mapping_fn": (
#                 lambda agent_id, episode, **kwargs: "shared_policy"),
#         },
#     },
# )

Visualize the results:

In [5]:
pass

### Central Environment with Stable-Baselines3

In [6]:
from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy

The **CentralMComWrapper** class can be used to wrap the default multi-agent setting. It simulates centralized control over user equipments (UEs), i.e., one single agent decides what connections should be established for each UE and is given global information. In practice, this assumption is unrealistic, but can simplify the learning procedure. 

The class wraps the observations, rewards and actions of the multi-agent setting. Now, observations are single (concatenated) vectors that jointly represent up-to-date information on all UEs. The reward is the average utility of active UEs. Similarly, actions are vectors of discrete decisions.

In [7]:
from mobile_env.wrappers.central import CentralMComWrapper

# wrap the multi-agent setting
env = gym.make("mobile-small-v0")
env = CentralMComWrapper(env)

Train stable-baselines3's PPO agent for 50000 time steps on the central environment:

In [ ]:
model = PPO(MlpPolicy, env, verbose=1)
model.learn(total_timesteps=50000)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1e+03    |
|    ep_rew_mean     | -453     |
| time/              |          |
|    fps             | 142      |
|    iterations      | 1        |
|    time_elapsed    | 14       |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1e+03       |
|    ep_rew_mean          | -416        |
| time/                   |             |
|    fps                  | 112         |
|    iterations           | 2           |
|    time_elapsed         | 36          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.015609202 |
|    clip_fraction        | 0.153       |
|    clip_range           | 0.2         |
|    entropy_loss   

Visualize what the central agent has learned:

In [10]:
obs = env.reset()
done = False
while not done:
    action, _ = model.predict(obs)
    obs, rew, info, done = env.step(action)
    env.render()

AssertionError: step() called on already terminated episode

### Sequential Action Wrapper for Central Environment with Stable-Baselines3

In [ ]:
from mobile_env.wrappers.sequential import SequentialCentralMComWrapper

<!-- The **SequentialCentralMComWrapper** class is an extension to the **CentralMComWrapper** class. It applies discrete actions in round-robin manner to all user equipments.  -->

In [ ]:
env = gym.make("mobile-small-v0")
env = SequentialCentralMComWrapper(env)

model = PPO(MlpPolicy, env, verbose=1)
model.learn(total_timesteps=1000)

obs = env.reset()
done = False
while not done:
    action, _ = model.predict(obs)
    obs, rew, info, done = env.step(action)
    env.render()

In [ ]:
env = gym.make("mobile-small-v0")
env = SequentialCentralMComWrapper(env)

env.reset()